In [1]:
import h5py

import numpy as np
import pandas as pd

from glob import glob
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
bay = linear_model.BayesianRidge(alpha_1 = 1e-06, alpha_2 = 1e-06, compute_score = False, copy_X = True,
   fit_intercept = True, lambda_1 = 1e-06, lambda_2 = 1e-06,
   normalize = False, tol=0.001, verbose = False)

## Explore all csv

In [3]:
# Age and 4 anonymized targets, 443 partially missed observations
train_scores = pd.read_csv('/kaggle/input/trends-assessment-prediction/train_scores.csv').sort_values(by='Id')

# Somehow preprocessed morphometry (after group ICA), simplest feature set
loadings = pd.read_csv('/kaggle/input/trends-assessment-prediction/loading.csv')

# resting-state fMRI Functional Network Connectivity matrices. 
# In simple setting, these are cross-correlations (in this case something more sophisticated) between
# every pair of brain regions presented in train/test *.mat
fnc = pd.read_csv('/kaggle/input/trends-assessment-prediction/fnc.csv')

# Submit Age and 4 scores for test ids
sample = pd.read_csv('/kaggle/input/trends-assessment-prediction/sample_submission.csv')

# List of some of subjects from test set whose data were collected from different scanner
reveal = pd.read_csv('/kaggle/input/trends-assessment-prediction/reveal_ID_site2.csv')

# 53 unique numbers between 2 and 99 (somehow related to brain regions? regions keys?)
icn_nums = pd.read_csv('/kaggle/input/trends-assessment-prediction/ICN_numbers.csv')

# Brain template
# /kaggle/input/trends-assessment-prediction/fMRI_mask.nii 

# train/test fMRI spatial maps
# *.mat

In [4]:
loadings.head(3)

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.006070,0.014466,0.004136,0.000658,-0.002742,0.005033,0.016720,0.003484,0.001797,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.009087,0.009291,0.007049,-0.002076,-0.002227,0.004605,0.012277,0.002946,0.004086,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,...,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164


In [5]:
# 53 * 52 / 2 = 1378 + Id column
fnc.head(3)

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
0,10001,0.368580,0.166876,0.438148,0.341007,-0.186251,0.049096,0.121417,-0.174268,-0.231578,...,-0.149279,0.552841,0.131046,0.335446,0.394867,-0.042853,0.124627,-0.060712,0.515964,0.290488
1,10002,0.151696,-0.024819,0.217504,0.418072,-0.227234,-0.064052,-0.143832,-0.118116,-0.054825,...,-0.214216,-0.039792,0.143014,-0.189962,0.498373,0.444231,0.592438,0.028649,0.705524,0.248327
2,10003,0.343415,0.109974,0.741641,0.578558,-0.676446,-0.436960,-0.295663,-0.377790,-0.344963,...,-0.154941,0.136850,-0.022361,0.137625,0.677972,0.409412,0.563892,0.438684,0.618204,0.284474


# Exploring fnc

In [6]:
import re
from tqdm import tqdm

In [7]:
r = re.compile('\d+')
col_dict = {}
for col in fnc.columns:
    ind = r.findall(col)
    if ind:
        col_dict[col] = [int(i) for i in ind]

In [8]:
def get_matrix(df_row, return_idx=False):
    matrix = np.zeros((100, 100))
    for col in df_row.index[1:]:
        i, j = col_dict[col]
        matrix[i, j] = df_row[col]
    matrix += matrix.T
    
    idx = np.array([ 2,  3,  4,  5,  7,  8,  9, 11, 12, 13, 15, 16, 17, 18, 20, 21, 23,
                     27, 32, 33, 37, 38, 40, 43, 45, 48, 51, 53, 54, 55, 56, 61, 62, 63,
                     66, 67, 68, 69, 70, 71, 72, 77, 79, 80, 81, 83, 84, 88, 93, 94, 96,
                     98, 99])
    if return_idx:
        return matrix[:, idx][idx, :], idx 
    return matrix[:, idx][idx, :]

In [9]:
degrees = []
for row in tqdm(fnc.iterrows()):
    mat = get_matrix(row[1])
    degrees.append(mat.sum(axis=1))

11754it [04:38, 42.21it/s]


In [10]:
_, idx = get_matrix(fnc.iloc[0], return_idx=True)
degrees = pd.DataFrame(degrees, columns=idx)
degrees['Id'] = fnc['Id']

In [11]:
len(glob('/kaggle/input/trends-assessment-prediction/fMRI_train/*.mat')), len(glob('/kaggle/input/trends-assessment-prediction/fMRI_test/*.mat'))

(5877, 5877)

In [12]:
sbj = glob('/kaggle/input/trends-assessment-prediction/fMRI_train/*.mat')[10]

with h5py.File(sbj, 'r') as f:
    mat = f['SM_feature'][()]
    mat = np.moveaxis(mat, [0,1,2,3], [3,2,1,0])
    
print(mat.shape)

(53, 63, 52, 53)


# Build a model

upd1. add mape_scorer

upd2. add fnc to features

upd3. add degrees features

upd4. add fcn/500 from https://www.kaggle.com/aerdem4/rapids-svm-on-trends-neuroimaging , switch to Ridge regression

todo: fit on all available targets (currently observation is dropped if any target is missing)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict

In [14]:
# train/test Ids
train_ids = sorted(loadings[loadings['Id'].isin(train_scores.Id)]['Id'].values)
test_ids = sorted(loadings[~loadings['Id'].isin(train_scores.Id)]['Id'].values)

# generate test DataFrame
test_prediction = pd.DataFrame(test_ids, columns=['Id'], dtype=str)

target_columns = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')
fnc_columns = fnc.columns[1:]
degrees_columns = degrees.columns[:-1]

# generate X, targets
data = pd.merge(loadings, train_scores, on='Id')#.dropna()
data = pd.merge(data, fnc, on='Id')
data = pd.merge(data, degrees, on='Id')

# X_train = data.drop(list(target_columns), axis=1).drop('Id', axis=1)
# y_train = data[list(target_columns)]

X_test = pd.merge(loadings[loadings.Id.isin(test_ids)], fnc, on='Id')
X_test = pd.merge(X_test, degrees, on='Id').drop('Id', axis=1)

## Implement mape scorer

Since lb uses weighted mape (all targets are > 0), we will implement mape scorer to pass into GridSearchCV

In [15]:
from sklearn.metrics import make_scorer

def MAPE(y_true, y_pred, **kwargs):
    '''Returns MAPE between y_true and y_pred'''
    return np.sum(np.abs(y_true - y_pred)) / y_true.sum()

mape_scorer = make_scorer(MAPE, greater_is_better=False)

In [16]:
# Setting up the model
# model = RandomForestRegressor(
#     max_depth=5,
#     min_samples_split=10,
#     min_samples_leaf=5
# )

# model = Lasso()
model = bay
# model = SVR()


cv = KFold(n_splits = 7, shuffle=True, random_state=29)

# grid = {
#     'max_depth':[2, 5, 10],
#     'n_estimators':[20, 30],
#     'max_features':[0.1, 0.2, 0.3, 0.5]
# }

#grid = {
#    'alpha': [0.0003, 0.001, 0.003, 0.01, 0.03,0.0001]
#}
grid = {'n_iter': [100,200,300]}
# grid = {
#     'C': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.5, 0.85, 1, 3, 5, 10]
# }

# grid = {
#     'alpha': np.linspace(0.0001, 0.001, 20)
# }
gs = GridSearchCV(model, grid, n_jobs=-1, cv=cv, verbose=0, scoring=mape_scorer)

In [17]:
# age 0.1446

# domain1_var1 0.1512

# domain1_var2 0.1513

# domain2_var1 0.1819

# domain2_var2 0.1763

In [18]:
# Training the model
best_models = {}
total_score = []

for col in target_columns:
    
    X_train = data.dropna(subset=[col], axis=0).drop(list(target_columns), axis=1).drop('Id', axis=1)
    X_train[fnc_columns] /= 600
    y_train = data.dropna(subset=[col], axis=0)[col]
    
    gs.fit(X_train, y_train)
    best_models[col] = gs.best_estimator_
    
    # Train performance
    y_pred = cross_val_predict(gs.best_estimator_, X_train, y_train, cv=cv, n_jobs=-1)
    total_score.append(MAPE(y_train, y_pred))
    print(col, MAPE(y_train, y_pred))

total_score = np.array(total_score)
print(f'Total score: {np.sum(total_score*[.3, .175, .175, .175, .175])}')

age 0.20919947562339652
domain1_var1 0.15804474007442154
domain1_var2 0.15135147941193514
domain2_var1 0.18630466379795924
domain2_var2 0.17919664682884168
Total score: 0.18086691045682152


In [19]:
def get_pred(col, model):
    X_train = data.dropna(subset=[col], axis=0).drop(list(target_columns), axis=1).drop('Id', axis=1)
    X_train[fnc_columns] /= 600
    y_train = data.dropna(subset=[col], axis=0)[col]
    
    # Train performance
    y_pred = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=-1)
    return y_pred

In [20]:
# Predicting test
X_test[fnc_columns] /= 600

for col in target_columns:
    test_prediction[col] = best_models[col].predict(X_test)

In [21]:
# Evaluate the lb metric on local cv

# def lb_metric(y_true, y_pred):
#     '''Computes lb metric, both y_true and y_pred should be DataFrames of shape n x 5'''
#     y_true = y_true[['age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2']]
#     y_pred = y_pred[['age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2']]
    
#     weights = np.array([.3, .175, .175, .175, .175])
#     return np.sum(weights * np.abs(y_pred.values - y_true.values).sum(axis=0) / y_train.values.sum(axis=0))

In [22]:
# train_prediction_cv = {}
# for col in target_columns:
#     train_prediction_cv[col] = cross_val_predict(best_models[col], X_train, y_train[col], cv = cv, n_jobs=-1)
# train_prediction_cv = pd.DataFrame(train_prediction_cv)

# lb_metric(y_train, train_prediction_cv)

## Making a submission

In [23]:
def make_sub(test_prediction):
    '''Converts 5877 x 6 DataFrame of predictions into 29385 x 2 DataFrame with valid Id'''
    target_columns = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')
    _columns = (0,1,2,3,4)
    tst = test_prediction.rename(columns=dict(zip(target_columns, _columns)))
    tst = tst.melt(id_vars='Id',
           value_vars=_columns,
           value_name='Predicted')

    tst['target_type'] = tst.variable.map(dict(zip(_columns, target_columns)))
    tst['Id_'] = tst[['Id', 'target_type']].apply(lambda x: '_'.join((str(x[0]), str(x[1]))), axis=1)

    return tst.sort_values(by=['Id', 'variable'])\
              .drop(['Id', 'variable', 'target_type'],axis=1)\
              .rename(columns={'Id_':'Id'})\
              .reset_index(drop=True)\
              [['Id', 'Predicted']]

In [24]:
sub = make_sub(test_prediction)

sub.head()

,Id,Predicted
0,10003_age,57.678344
1,10003_domain1_var1,53.465309
2,10003_domain1_var2,59.237550
3,10003_domain2_var1,48.128446
4,10003_domain2_var2,52.341559


In [25]:
sub.to_csv('ridge_mape_500.csv', index=False)